# Stage 1: Context Builder - Unit Tests

This notebook tests the **Context Builder** service in isolation.

**What it tests:**
- Company info extraction (mock)
- Temporal context extraction
- Persona extraction (RBC & client)
- User overrides
- Error handling

In [ ]:
# Setup path
import sys
sys.path.insert(0, "../..")

from cmpt.services import ContextBuilderService, ChainRequest, ChainRequestOverrides
from cmpt.services.models import ContextBuilderOutput

## Test 1: Basic Company Extraction

In [ ]:
async def test_basic_company_extraction():
    """Test basic company info extraction."""
    service = ContextBuilderService()
    
    request = ChainRequest(
        corporate_company_name="Apple Inc",
        meeting_datetime="2025-01-15T10:00:00Z",
    )
    
    output = await service.execute(request)
    
    # Verify output
    assert output.company_name == "Apple Inc"
    assert output.company_info is not None
    assert output.company_info.name == "Apple Inc"
    print(f"✓ Company: {output.company_name}")
    print(f"✓ Ticker: {output.ticker}")
    print(f"✓ Industry: {output.company_info.industry}")
    return output

output = await test_basic_company_extraction()

## Test 2: Temporal Context Extraction

In [ ]:
async def test_temporal_context():
    """Test temporal context extraction."""
    service = ContextBuilderService()
    
    request = ChainRequest(
        corporate_company_name="Microsoft",
        meeting_datetime="2025-03-15T14:00:00Z",  # Q1 meeting
    )
    
    output = await service.execute(request)
    
    # Verify temporal context
    assert output.temporal_context is not None
    assert output.temporal_context.fiscal_quarter == "1"  # Q1
    assert output.temporal_context.fiscal_year == "2025"
    
    print(f"✓ Meeting Date: {output.temporal_context.meeting_date}")
    print(f"✓ Fiscal Quarter: Q{output.temporal_context.fiscal_quarter}")
    print(f"✓ Fiscal Year: {output.temporal_context.fiscal_year}")
    print(f"✓ News Lookback: {output.temporal_context.news_lookback_days} days")
    return output

output = await test_temporal_context()

## Test 3: RBC Persona Extraction

In [ ]:
async def test_rbc_persona():
    """Test RBC employee persona extraction."""
    service = ContextBuilderService()
    
    request = ChainRequest(
        corporate_company_name="Tesla",
        meeting_datetime="2025-02-20T10:00:00Z",
        rbc_employee_email="john.smith@rbc.com",
    )
    
    output = await service.execute(request)
    
    # Verify RBC persona
    assert output.rbc_persona is not None
    assert output.rbc_persona.email == "john.smith@rbc.com"
    assert output.rbc_persona.is_internal is True
    
    print(f"✓ RBC Employee: {output.rbc_persona.name}")
    print(f"✓ Email: {output.rbc_persona.email}")
    print(f"✓ Source: {output.rbc_persona.source}")
    return output

output = await test_rbc_persona()

## Test 4: Client Persona Extraction

In [ ]:
async def test_client_persona():
    """Test corporate client persona extraction."""
    service = ContextBuilderService()
    
    request = ChainRequest(
        corporate_company_name="Amazon",
        meeting_datetime="2025-02-20T10:00:00Z",
        corporate_client_email="jane.doe@amazon.com",
    )
    
    output = await service.execute(request)
    
    # Verify client persona
    assert len(output.corporate_client_personas) > 0
    client = output.corporate_client_personas[0]
    assert client.email == "jane.doe@amazon.com"
    assert client.is_internal is False
    
    print(f"✓ Client: {client.name}")
    print(f"✓ Company: {client.company}")
    print(f"✓ Source: {client.source}")
    return output

output = await test_client_persona()

## Test 5: User Overrides

In [ ]:
async def test_user_overrides():
    """Test that user overrides take precedence."""
    service = ContextBuilderService()
    
    request = ChainRequest(
        corporate_company_name="Google",
        meeting_datetime="2025-04-15T10:00:00Z",
        overrides=ChainRequestOverrides(
            ticker="GOOGL",  # Override ticker
            fiscal_quarter="Q2",  # Override quarter
            news_lookback_days=60,  # Custom lookback
        ),
    )
    
    output = await service.execute(request)
    
    # Verify overrides were applied
    assert output.company_info.ticker == "GOOGL"
    assert output.temporal_context.fiscal_quarter == "2"
    assert output.temporal_context.news_lookback_days == 60
    
    print(f"✓ Ticker Override: {output.company_info.ticker}")
    print(f"✓ Quarter Override: Q{output.temporal_context.fiscal_quarter}")
    print(f"✓ Lookback Override: {output.temporal_context.news_lookback_days} days")
    return output

output = await test_user_overrides()

## Test 6: Error Handling

In [ ]:
async def test_error_handling():
    """Test graceful error handling."""
    service = ContextBuilderService(
        extractor_timeouts={"firm": 0.001}  # Very short timeout to force error
    )
    
    request = ChainRequest(
        corporate_company_name="TestCorp",
        meeting_datetime="2025-01-01T10:00:00Z",
    )
    
    # Should not raise, should capture errors
    output = await service.execute(request)
    
    # Output should still be returned
    assert output is not None
    print(f"✓ Output returned despite errors")
    print(f"✓ Errors captured: {list(output.errors.keys())}")
    return output

output = await test_error_handling()

## Test 7: Timing Metrics

In [ ]:
async def test_timing_metrics():
    """Test that timing metrics are captured."""
    service = ContextBuilderService()
    
    request = ChainRequest(
        corporate_company_name="Netflix",
        meeting_datetime="2025-01-20T10:00:00Z",
        rbc_employee_email="analyst@rbc.com",
    )
    
    output = await service.execute(request)
    
    # Verify timing metrics exist
    assert output.timing_ms is not None
    assert "total" in output.timing_ms
    
    print("✓ Timing Metrics:")
    for key, value in output.timing_ms.items():
        print(f"    {key}: {value:.2f}ms")
    return output

output = await test_timing_metrics()

## Summary

In [ ]:
print("="*60)
print("  Stage 1: Context Builder - All Tests Passed!")
print("="*60)
print()
print("Tested:")
print("  ✓ Basic company extraction")
print("  ✓ Temporal context extraction")
print("  ✓ RBC persona extraction")
print("  ✓ Client persona extraction")
print("  ✓ User overrides")
print("  ✓ Error handling")
print("  ✓ Timing metrics")